# Chapter 3_텐서플로우 기본

### simple model
<img src="./img/simple.jpeg" width=50%>

In [1]:
import tensorflow as tf
import numpy as np
import os

In [11]:
a = tf.constant(5, name="imput_a")
b = tf.constant(3, name="imput_b")

In [12]:
c = tf.multiply(a,b, name="mul_c")
d = tf.add(a,b, name="add_d")

In [13]:
e = tf.add(c,d, name="add_e")

In [16]:
sess = tf.Session()
sess.run(e)

23

### 개선사항
- 입력은 tf.constant 노드 대신에 __플레이스홀더를 사용.__
- 두개의 이산 스칼라(discrete scalar) 입력 대신에 __가변 길이의 하나의 벡터 사용.__
- 그래프를 사용하는 동안의 시간에 따른 __모든 출력값을 누적.__
- 그래프를 __namescope__로 깔끔하게 분할.
- 각각의 실행 후에 그래프의 출력, 모든 출력 결과의 누적, 모든 출력 결과의 평균을 __텐서보드에서 사용하기 위해서 디스크에 저장.__

### better model
<img src="./img/better.jpeg" width=50%>

In [49]:
### 그래프 만들기

graph = tf.Graph()
with graph.as_default():
    
    ### 속성상 전역변수들인 것들 
    with tf.name_scope("variables"): 
        global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name="global_step") 
        total_output = tf.Variable(0.0, dtype=tf.float32, trainable=False, name="global_step")
        # trainable=False : 변수들이 수작업으로 셋팅될것이라고 명시
        
    ### 핵심 연산 부분    
    with tf.name_scope("transformation"): 
        
        # input layer
        with tf.name_scope("input"):
            a = tf.placeholder(tf.float32, shape=[None], name="input_placeholder_a")
            
        # middle layer
        with tf.name_scope("intermediate_layer"):
            b = tf.reduce_prod(a, name="plod_b")
            c = tf.reduce_sum(a, name="sum_c")
            
        # output layer
        with tf.name_scope("output"):
            output = tf.add(b,c, name="output")
            
            
    ### transformation 연산을 마친후에 두개의 변수를 업데이트하는 연산
    with tf.name_scope("update"):
        # 최근 output으로부터 total_output 변수 증가.
        update_total = total_output.assign_add(output)
        
        # 그래프가 run할때마다 global_step 변수 증가.
        increment_step = global_step.assign_add(1)
        
        
    ### 텐서보드 요약
    with tf.name_scope("summaries"):
        avg = tf.div(update_total, tf.cast(increment_step, tf.float32), name="average")
        tf.summary.scalar('Output', output)
        tf.summary.scalar('Sum_of_outputs_over_time', update_total)
        tf.summary.scalar('Average_of_outputs_over_time', avg)
        
#         tf.summary.scalar('Output', output, name="output_summary")
#         tf.summary.scalar('Sum of outputs over time', update_total, name="total_summary")
#         tf.summary.scalar('Average of outputs over time', avg, name="average_summary")
        
    ### 변수 초기화와 모든 summary를 합치기 위한 helper node를 하나의 operation으로 만듦.
    with tf.name_scope("global_ops"):
        # Op 초기화
        init = tf.global_variables_initializer()
        
        # 모든 summary를 하나의 operation으로 결합.
        ## 왜 위summaries에 안넣고 여기에 놓냐? -> 전역 operation과 같이 두는 것이 일반적으로 좋은 방법. 나중에 summary가 여기저기 있더라도 여기서 관리하면 됨.
        merged_summaries = tf.summary.merge_all()
        

In [50]:
sess = tf.Session(graph=graph)
writer = tf.summary.FileWriter('./improved_graph', graph)

In [51]:
sess.run(init)

In [52]:
def run_graph(input_tensor):
    feed_dict = {a: input_tensor}
    
    # session이 그래프를 실행. (아래 3개의 ops를 실행.)
    _output, step, summary = sess.run([output, increment_step, merged_summaries], feed_dict=feed_dict)
    
    print(_output, step)
    
    # summary를 summarywriter에 더함.
    writer.add_summary(summary, global_step=step)

In [56]:
run_graph([1,100])

201.0 4


In [57]:
# summary를 디스트에 씀.
writer.flush()

In [58]:
writer.close()
sess.close()

# Chapter 4_머신러닝 기초


- 지도학습 템플릿

In [ ]:
### 1. variable, model parameter 초기화

### 2. training loop operation 정의
def inference(X):
    # 추론 모델 계산
    pass

def loss(X, Y):
    # 예상출력과 레이블된 겂의 비교를 통해 손실 계산
    pass

def inputs():
    # 학습 데이터 읽음
    pass

def train(total_loss):
    # 계산된 loss에 대해 모델 파라미터를 갱신
    pass
        
def evaluate(sess, X, Y):
    # 학습된 모델을 통해 평가
    pass

### 3. 그래프 런칭
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    X, Y = inputs()
    
    total_loss = loss(X, Y)
    train_op = train(total_loss)
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    # actual training loop
    training_steps = 1000
    
    for step in range(training_steps):
        sess.run([train_op])
        if step % 10 == 0:
            print("{} step // loss: {}".format(step, sess.run([total_loss])))
            
    evaluate(sess, X, Y)
    
    coord.request_stop()
    coord.join(threads)

### 선형회귀

In [45]:
### 1. variable, model parameter 초기화
W = tf.Variable(tf.zeros([2,1]), name='weights')
b = tf.Variable(0., name='bias')


### 2. training loop operation 정의
def inference(X):
    # 추론 모델 계산
    return tf.matmul(X,W) + b

def loss(X, Y):
    # 예상출력과 레이블된 겂의 비교를 통해 손실 계산
    Y_predicted = inference(X)
    return tf.reduce_sum(tf.squared_difference(Y, Y_predicted))

def inputs():
    # 학습 데이터 읽음
    weight_age = [[84, 46], [73, 20], [65, 52], [70, 30], [76, 57], [69, 25], [63, 28], [72, 36], 
                  [79, 57], [75, 44], [27, 24], [89, 31], [65, 52], [57, 23], [59, 60], [69, 48], 
                  [60, 34], [79, 51], [75, 50], [82, 34], [59, 46], [67, 23], [85, 37], [55, 40], [63, 30]]
    
    blood_fat_content = [354, 190, 405, 263, 451, 302, 288, 385, 402, 365, 209, 290, 346, 254, 395, 
                         434, 220, 374, 308, 220, 311, 181, 274, 303, 244]
    return tf.to_float(weight_age), tf.to_float(blood_fat_content)

def train(total_loss):
    # 계산된 loss에 대해 모델 파라미터를 갱신
    learning_rate = 0.00000001
    return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)
        
def evaluate(sess, X, Y):
    # 학습된 모델을 통해 평가
    print(sess.run(inference([[80., 25.]])))  # ~ 303
    print(sess.run(inference([[65., 25.]])))  # ~ 256

### 3. 그래프 런칭
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    X, Y = inputs()
    
    total_loss = loss(X, Y)
    train_op = train(total_loss)
    
    # actual training loop
    training_steps = 100
    
    for step in range(training_steps):
        sess.run([train_op])
        if step % 10 == 0:
            print("{} step // loss: {}".format(step, sess.run([total_loss])))
            
    evaluate(sess, X, Y)

0 step // loss: [54929292.0]
10 step // loss: [14629747.0]
20 step // loss: [7090800.0]
30 step // loss: [5680201.0]
40 step // loss: [5416011.0]
50 step // loss: [5366280.0]
60 step // loss: [5356678.0]
70 step // loss: [5354588.0]
80 step // loss: [5353913.5]
90 step // loss: [5353510.0]
[[ 313.60101318]]
[[ 263.45443726]]


### 로지스틱회귀

In [2]:
### 1. variable, model parameter 초기화
W = tf.Variable(tf.zeros([5, 1]), name="weights")
b = tf.Variable(0., name="bias")

### 2. training loop operation 정의
def combine_inputs(X):
    # 입력값을 하나로 결합.
    # 앞에서의 inference가 입력값 결합을 위해 사용됨.
    return tf.matmul(X, W) + b

def inference(X):
    # 추론 모델 계산
    return tf.sigmoid(combine_inputs(X))

def loss(X, Y):
    # 예상출력과 레이블된 겂의 비교를 통해 손실 계산
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(combine_inputs(X),Y))

def read_csv(batch_size, file_name, record_defaults):
    filename_queue = tf.train.string_input_producer([os.path.join(os.getcwd(), file_name)])

    reader = tf.TextLineReader(skip_header_lines=1)
    key, value = reader.read(filename_queue)

    # decode_csv will convert a Tensor from type string (the text line) in
    # a tuple of tensor columns with the specified defaults, which also
    # sets the data type for each column
    decoded = tf.decode_csv(value, record_defaults=record_defaults)

    # batch actually reads the file and loads "batch_size" rows in a single tensor
    return tf.train.shuffle_batch(decoded,
                                  batch_size=batch_size,
                                  capacity=batch_size * 50,
                                  min_after_dequeue=batch_size)


def inputs():
    # 학습 데이터 읽음
    passenger_id, survived, pclass, name, sex, age, sibsp, parch, ticket, fare, cabin, embarked = \
        read_csv(100, "./data/train.csv", [[0.0], [0.0], [0], [""], [""], [0.0], [0.0], [0.0], [""], [0.0], [""], [""]])

    # convert categorical data
    is_first_class = tf.to_float(tf.equal(pclass, [1]))
    is_second_class = tf.to_float(tf.equal(pclass, [2]))
    is_third_class = tf.to_float(tf.equal(pclass, [3]))

    gender = tf.to_float(tf.equal(sex, ["female"]))

    # Finally we pack all the features in a single matrix;
    # We then transpose to have a matrix with one example per row and one feature per column.
    features = tf.transpose(tf.pack([is_first_class, is_second_class, is_third_class, gender, age]))
    survived = tf.reshape(survived, [100, 1])

    return features, survived

def train(total_loss):
    # 계산된 loss에 대해 모델 파라미터를 갱신
    learning_rate = 0.01
    return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)
        
def evaluate(sess, X, Y):
    # 학습된 모델을 통해 평가
    predicted = tf.cast(inference(X) > 0.5, tf.float32)
    print(sess.run(tf.reduce_mean(tf.cast(tf.equal(predicted, Y), tf.float32))))

In [6]:
### 3. 그래프 런칭
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    X, Y = inputs()
    
    total_loss = loss(X, Y)
    train_op = train(total_loss)
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    # actual training loop
    training_steps = 1000
    
    for step in range(training_steps):
        sess.run([train_op])
        if step % 100 == 0:
            print("{} step // loss: {}".format(step, sess.run([total_loss])))
            
    evaluate(sess, X, Y)
    coord.request_stop()
    coord.join(threads)

0 step // loss: [0.68657357]
100 step // loss: [0.65677369]
200 step // loss: [0.77893537]
300 step // loss: [0.80339158]
400 step // loss: [0.59002984]
500 step // loss: [0.57293725]
600 step // loss: [0.70559955]
700 step // loss: [0.63009131]
800 step // loss: [0.79282361]
900 step // loss: [0.5779621]
0.76
